In [ ]:
#!jupyter nbconvert --to=python CO2flux_TotalGlobal_python3.ipynb

In [ ]:
class CO2flux_TotalGlobal:   
    '''
    class CO2flux_TotalGlobal(runname,resultpath,savepath,meshpath,first_year,last_year,
                 mapproj='pc',savefig=False,layerwise=False)
    '''
    def __init__(self,runname,resultpath,savepath,meshpath,first_year,last_year,
                 mapproj='pc',
                 savefig=False,
                 layerwise=False,depth_array=[]):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.meshpath = meshpath
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf

        from plot_Taylor_normalized import plt_Taylor_norm

        # load FESOM mesh -------------------------------------------------------------------------------------
        mesh       = pf.load_mesh(meshpath)
        years = range(self.fyear, self.lyear+1)
        
        
        # load nodal area -------------------------------------------------------------------------------------
        meshdiag=resultpath+'/'+self.runname+'.mesh.diag.nc'
        #!ncdump -h $meshdiag
        diag = pf.get_meshdiag(mesh,meshdiag=meshdiag, runid=self.runname)   
        
        nod_area = diag.rename_dims({"nl": "nz1", "nod_n": "nod2"}).nod_area
        nod_area.load()
        
        # get only uppermost layer, mask where equal to zero
        nod_area_surface = np.ma.masked_equal(nod_area[0, :].data, 0)
        
        ## same as FESOM1.4 code??
        #f           = Dataset(fesom.initial.mesh.diag.nc, 'r')
        #NodalArea2D = f.variables['cluster_area'][:]
        ## cluster_area:description = "2D mesh cluster area" ;
        ## cluster_area:units = "m^2" ;
        
        # calculating total NPP per year -------------------------------------------------------------------------------
        CO2flux = np.zeros(len(years))
            
        for i in range(len(years)):
            ind_year = years[i]
            print('loading {0}'.format(ind_year))
            
            flux = pf.get_data(resultpath, "CO2f", ind_year, mesh, how="mean", compute=True, 
                               runid=self.runname, silent=True)
            # CO2f:description = "CO2-flux into the surface water" 
            # CO2f:units = "mmolC/m2/d"
            #print(np.shape(flux))
            flux        = 12.01 * flux * 365
            CO2flux[i] = np.sum(flux*nod_area_surface)/1e18 # summing up and converting to [Pg C/year]

        
        # plotting CO2 flux -------------------------------------------------------------------------------        
        
        fig = plt.figure(figsize=(8, 8), facecolor='w', edgecolor='k')
        plt.plot(years,CO2flux,'.-', label='FESOM')
        plt.title(self.runname+': CO$_2$ flux into surface water')
        plt.ylabel(r'[Pg C yr$^{-1}$]')
        #plt.legend(loc='lower left')
        #plt.ylim(-2,2)
        
        if(self.savefig == True):
            plt.savefig(self.savepath+self.runname+'_CO2flux_TotalGlobal'+'_'+str(self.fyear)+'to'+str(self.lyear)+'.png', 
                    dpi = 300, bbox_inches='tight')
            
        plt.show(block=False)

In [ ]:
### TESTING ###
if __name__ == "__main__":  
    # run specification -------------------------------------------------------------------------------------
    runid      =  'mo1'
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + runid
    savepath = '/home/ollie/mozeisin/evaluation/mo_files/'

    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

    # period of analysis ------------------------------------------------------------------------------------
    first_year = 1948
    last_year  = 1950
    
    # now test:runname,resultpath,savepath,meshpath,first_year,last_year
    test = CO2flux_TotalGlobal(runid,resultpath,savepath,meshpath,first_year,last_year,layerwise=False,savefig=False)

In [ ]:
### TESTING ###
if __name__ == "__main__": 
    
    # necessary modules -------------------------------------------------------------------------------------
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm

    # run specification -------------------------------------------------------------------------------------
    runid      =  'mo1'
    layerwise    = False

    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + runid

    # mesh initialization -----------------------------------------------------------
    mesh = pf.load_mesh(meshpath)
    meshdiag=resultpath+'/'+runid+'.mesh.diag.nc'
    #!ncdump -h $meshdiag

    diag = pf.get_meshdiag(mesh,meshdiag=meshdiag, runid=runid)
    print(diag)

    # get area
    nod_area = diag.rename_dims({"nl": "nz1", "nod_n": "nod2"}).nod_area
    nod_area.load()
    #print(np.shape(nod_area[0,:]), type(nod_area))
    
    pf.plot(mesh,np.array(nod_area[0,:]),units='Nodal area', mapproj='pc')

    # overview original ncfiles
    #!ncdump -h '/work/ollie/mozeisin/results/f2r1.2/mo1/CO2f.mo1.1948.nc'
    #!ncdump -h '/work/ollie/mozeisin/results/f2r1.2/mo1/mo1.mesh.diag.nc'